In [ ]:
import sys
print(sys.executable)
sys.path.append('/home/richard/profitviews/crypto-stock')

In [2]:
from my import venues

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

# `dotenv` is a package that loads environment variables from a `.env` file into the environment.
# It is used to store sensitive information like API keys in a file that is not checked into version control.
# The `.env` file is not checked into version control, so it is not included in the repository.
# The `.env` file is created by the user and contains the API keys for the various services used in the project.
# See https://pypi.org/project/python-dotenv/ for more information.

env_path = Path.home() / 'prod-bot' / '.env'  # Adjust this path as needed
print(env_path)
load_dotenv(env_path)

In [4]:
ALPACA_API_KEY=os.getenv('ALPACA_API_KEY')
ALPACA_API_SECRET=os.getenv('ALPACA_API_SECRET')
ALPACA_PAPER_API_KEY=os.getenv('ALPACA_PAPER_API_KEY')
ALPACA_PAPER_API_SECRET=os.getenv('ALPACA_PAPER_API_SECRET')

In [ ]:
w = venues.Alpaca(
	None, ALPACA_PAPER_API_KEY, ALPACA_PAPER_API_SECRET, 
	# trading_endpoint='https://api.alpaca.markets', 
	data_endpoint='https://data.alpaca.markets')

In [ ]:
w.mark_price('IBIT')

In [10]:
w.standard_size('IBIT', 100)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): data.alpaca.markets:443


Fetching quote from: https://data.alpaca.markets/v2/stocks/IBIT/quotes/latest
send: b'GET /v2/stocks/IBIT/quotes/latest HTTP/1.1\r\nHost: data.alpaca.markets\r\nUser-Agent: python-requests/2.32.3\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nAPCA-API-KEY-ID: PK1D24H7941XGBR417QH\r\nAPCA-API-SECRET-KEY: OCa5hMZvColZ8r1oQ6BxTq0heCoEyubkxPDXZgY5\r\n\r\n'


DEBUG:urllib3.connectionpool:https://data.alpaca.markets:443 "GET /v2/stocks/IBIT/quotes/latest HTTP/1.1" 200 137


reply: 'HTTP/1.1 200 OK\r\n'
header: Date: Wed, 26 Feb 2025 15:39:46 GMT
header: Content-Type: application/json; charset=UTF-8
header: Content-Length: 137
header: Connection: keep-alive
header: X-Feed: iex
header: X-Ratelimit-Limit: 200
header: X-Ratelimit-Remaining: 199
header: X-Ratelimit-Reset: 1740584386
header: Strict-Transport-Security: max-age=31536000; includeSubDomains
header: X-Request-ID: d99d0cf4f136790f3d313f21ad69358d
header: Access-Control-Allow-Origin: *
header: Access-Control-Allow-Credentials: true
header: Access-Control-Allow-Methods: GET, OPTIONS
header: Access-Control-Allow-Headers: Apca-Api-Key-Id, Apca-Api-Secret-Key, Authorization
header: Access-Control-Max-Age: 1728000


2

In [ ]:
try:
    url = w.pricing_endpoint.format(symbol='IBIT')
    response = requests.get(url, headers=w.HEADERS)
    response.raise_for_status()
    quote = response.json()
    print("Latest quote for IBIT:", quote)
except Exception as e:
    print("Error fetching latest quote for IBIT:", e)


In [ ]:
try:
    ibit_quote_url = w.pricing_endpoint.format(symbol="IBIT")
    print("Fetching IBIT quote from:", ibit_quote_url)
    response = requests.get(ibit_quote_url, headers=w.HEADERS)
    response.raise_for_status()
    ibit_quote = response.json()
    print("IBIT Quote:", ibit_quote)
except Exception as e:
    print("Error fetching IBIT quote:", e)


In [ ]:
# Python code to calculate implied Bitcoin price from IBIT share price

def calculate_implied_btc_price(share_price, total_btc_held, total_shares_outstanding):
    """
    Calculate the implied Bitcoin price from IBIT share price.
    
    Args:
        share_price (float): Current IBIT share price in USD
        total_btc_held (float): Total Bitcoin held by the IBIT trust
        total_shares_outstanding (float): Total number of IBIT shares outstanding
    
    Returns:
        float: Implied Bitcoin price in USD
    """
    # Calculate the Bitcoin per Share Ratio
    btc_per_share_ratio = total_btc_held / total_shares_outstanding
    
    # Calculate the implied Bitcoin price
    implied_btc_price = share_price / btc_per_share_ratio
    
    return implied_btc_price

# Example values based on February 26, 2025 estimate
ibit_share_price = 50.00  # Estimated IBIT share price in USD (Feb 25 X post)
total_btc_held = 587698  # Total Bitcoin held by IBIT (as of Feb 20, 2025)
total_shares_outstanding = 1034000000  # Total shares outstanding (1.034 billion, Feb 20, 2025)

# Calculate the implied Bitcoin price
implied_price = calculate_implied_btc_price(ibit_share_price, total_btc_held, total_shares_outstanding)

# Print the result
print(f"IBIT Share Price: ${ibit_share_price:.2f}")
print(f"Bitcoin per Share Ratio: {total_btc_held / total_shares_outstanding:.7f} BTC/share")
print(f"Implied Bitcoin Price: ${implied_price:,.2f} USD/BTC")

# Optional: Compare with a known BTC price for sanity check (e.g., Feb 25 estimate)
current_btc_price = 87000  # Approximate BTC price from X post on Feb 25
print(f"Estimated Market Bitcoin Price: ${current_btc_price:,.2f} USD/BTC")
print(f"Difference (Premium/Discount): ${(implied_price - current_btc_price):,.2f} USD/BTC")